In [33]:
# BlochainUtils.py
from Crypto.Hash import SHA256
import json

class BlockchainUtils():

    @staticmethod
    def hash(data):
        data_string = json.dumps(data)
        data_bytes = data_string.encode("utf-8")
        data_hash = SHA256.new(data_bytes)
        return data_hash

In [34]:
# Transaction.py
import uuid
import time
from copy import deepcopy

class Transaction():

    def __init__(self, sender_public_key, receiver_public_key, amount, tx_type):
        self.sender_public_key = sender_public_key
        self.receiver_public_key = receiver_public_key
        self.amount = amount
        self.tx_type = tx_type
        self.id = (uuid.uuid1()).hex
        self.timestamp = time.time()
        self.signature = ""

    def to_json(self):
        return self.__dict__

    def sign(self, signature):
        self.signature = signature

    def payload(self):
        json_representation = deepcopy(self.to_json())
        json_representation["signature"] = ""
        return json_representation

    def equals(self, transaction):
        if self.id == transaction.id:
            return True
        else:
            return False
        

In [35]:
# Wallet.py
from Crypto.PublicKey import RSA
# from Transaction import Transaction
#from BlockchainUtils import BlockchainUtils
from Crypto.Signature import PKCS1_v1_5

class Wallet():

    def __init__(self):
        self.key_pair = RSA.generate(2048)

    def sign(self, data):
        data_hash = BlockchainUtils.hash(data)
        signature_scheme_object = PKCS1_v1_5.new(self.key_pair)
        signature = signature_scheme_object.sign(data_hash)
        return signature.hex()

    @staticmethod
    def signature_valid(data, signature, public_key_string):
        signature = bytes.fromhex(signature)
        data_hash = BlockchainUtils.hash(data)
        public_key = RSA.importKey(public_key_string)
        signature_scheme_object = PKCS1_v1_5.new(public_key)
        signature_valid = signature_scheme_object.verify(data_hash, signature)
        return signature_valid

    def public_key_string(self):
        public_key_string = self.key_pair.publickey().exportKey('PEM').decode("utf8")
        return public_key_string

    def create_tx(self, receiver, amount, tx_type):
        transaction = Transaction(self.public_key_string(), receiver, amount, tx_type)
        signature = self.sign(transaction.payload())
        transaction.sign(signature)
        return transaction

In [36]:
# TransactionPool.py

class TransactionPool():

    def __init__(self):
        self.transactions = []

    def add_tx(self, tx):
        self.transactions.append(tx)

    def tx_exists(self, tx):
        for pool_tx in self.transactions:
            if pool_tx.equals(tx):
                return True
        return False

In [37]:
# para probar código
# main.py

#from Transaction import Transaction
#from Wallet import Wallet
#from TransactionPool import TransactionPool

sender = "gustavo"
receiver = "maría"
amount = 300
tx_type = "TRANSFER"

wallet = Wallet()
fraudulent_wallet = Wallet()
pool = TransactionPool()

In [38]:
tx = wallet.create_tx(receiver, amount, tx_type)

In [39]:
tx.to_json()

{'sender_public_key': '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAruUPk0j2OniLBE0sjZ8J\n+uQytEuE/kxFO1UPgiKrQclYiR+KsCS4v5S5EPCaCSZgJDwR0LBhCX34lZC3ikje\n7V5jZ+RM+2lzFvEU2DpizorbEOjfsePObEGP7s/v4pWu48+E2mVcMuNazchLlmfS\n63aPhB/8PROvskExo6rpgVcqZuOwZs+v766blDq7zFxJXmk8S71lFzTf/cnVDpE6\ngGBaiaC00q8VjwlHrY4wdWsfymD+jeNX57I2nFr28Mv0axA4FbNTjrTUGxCrkrco\nOrRiA+K++JvqF1UcMe0Pr/Mn/7m1llSOe8zorCrLg+grcnqFy62fZ6m0VBpm353l\niwIDAQAB\n-----END PUBLIC KEY-----',
 'receiver_public_key': 'maría',
 'amount': 300,
 'tx_type': 'TRANSFER',
 'id': 'e3ae5051e88811efa554fcb3bcfce7a6',
 'timestamp': 1739286010.4823148,
 'signature': '8cf651191da517546af7e7432f11b3c54e7c8910664763052ac70069204bb5404adace97475db37698bd019ec40281fdb0bcb7f70fb92c9a6202f8554a2129e5bbca5229a07d828459c537ff22ad6fc2792cfc94a8d71cafa124b23c9386506389ebabb79bd66f0df1ac8b6b085a64bf952e138ce450f064a9661f077f5c889e9154718aa5c1ac936423e640613719a82de9772259b8bfa299bdb8bb51d973cbd6bc58838cd7d90d2cdf8bfe1651b0a

In [40]:
fraudulent_tx = fraudulent_wallet.create_tx(receiver, amount, tx_type)

In [41]:
fraudulent_tx.to_json()

{'sender_public_key': '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA2CA1ltVYbWy5xAGAXJl6\n9F0AePvZ+mDuuL2qRN35VI8viyjgQaOTh7VoyT5f/ZazykaGGOaOA98izarNUczd\nIm1RB2sNQF/j2cHcrpLHRwUciTeP4eNJeI8emun1qE3TDPhmZ6dgR/EWUDKlcHCi\nNFHYFcYQ5NeV4AjO6jZwCygbMyKA69n0sfFfck1oN47aZ3Lahv/nYJC0hEDvvIUE\nOL1oRu/YuavOb2W/xL/TIJsdVlzjQF+YWh/JKdLjFVTm/5RdeOWUgLO2uqIVJ+vl\ndELmEhFN/Sjztt6xP0U/wou70eVbvSmXbZXHlFrIC7XxDzq7ivYyc7XagMGw7nyl\ntQIDAQAB\n-----END PUBLIC KEY-----',
 'receiver_public_key': 'maría',
 'amount': 300,
 'tx_type': 'TRANSFER',
 'id': 'e3b472e7e88811efb576fcb3bcfce7a6',
 'timestamp': 1739286010.5225232,
 'signature': '63366ea6a074a560b6acd5a72911b9a77911fa6fa0d37571febff6dff3cb3544b0e2883a3a0428de951ff2f55efb408be9d4fc05a9c24befe4d962c1307235409e15c6290c4814c3cf165822b3f55454e9bfb21961e5c0e5c6fe584347566abe959f556b8cfb24863dae25be48900c51166fc4ad4b0b06cb679746503fa3931db2e0d1e6522eee575c83eab565958135a329ac6f6f46c44ed851f4416f21839666ac2bc16853087675fe3af71d3113a

In [42]:
signature_valid = Wallet.signature_valid(tx.payload(), tx.signature, fraudulent_wallet.public_key_string())

In [43]:
signature_valid

False

In [44]:
print(pool.transactions)

[]


In [45]:
if not pool.tx_exists(tx):
    pool.add_tx(tx)
print(pool.transactions)

In [46]:
if not pool.tx_exists(tx):
    pool.add_tx(tx)
print(pool.transactions)

In [47]:
tx_2 = wallet.create_tx(receiver, amount, tx_type)

In [48]:
if not pool.tx_exists(tx_2):
    pool.add_tx(tx_2)
print(pool.transactions)

[<__main__.Transaction object at 0x000001DC028C4D70>, <__main__.Transaction object at 0x000001DC027AF250>]


In [49]:
for pool_tx in pool.transactions:
    print(pool_tx.to_json())

{'sender_public_key': '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAruUPk0j2OniLBE0sjZ8J\n+uQytEuE/kxFO1UPgiKrQclYiR+KsCS4v5S5EPCaCSZgJDwR0LBhCX34lZC3ikje\n7V5jZ+RM+2lzFvEU2DpizorbEOjfsePObEGP7s/v4pWu48+E2mVcMuNazchLlmfS\n63aPhB/8PROvskExo6rpgVcqZuOwZs+v766blDq7zFxJXmk8S71lFzTf/cnVDpE6\ngGBaiaC00q8VjwlHrY4wdWsfymD+jeNX57I2nFr28Mv0axA4FbNTjrTUGxCrkrco\nOrRiA+K++JvqF1UcMe0Pr/Mn/7m1llSOe8zorCrLg+grcnqFy62fZ6m0VBpm353l\niwIDAQAB\n-----END PUBLIC KEY-----', 'receiver_public_key': 'maría', 'amount': 300, 'tx_type': 'TRANSFER', 'id': 'e3ae5051e88811efa554fcb3bcfce7a6', 'timestamp': 1739286010.4823148, 'signature': '8cf651191da517546af7e7432f11b3c54e7c8910664763052ac70069204bb5404adace97475db37698bd019ec40281fdb0bcb7f70fb92c9a6202f8554a2129e5bbca5229a07d828459c537ff22ad6fc2792cfc94a8d71cafa124b23c9386506389ebabb79bd66f0df1ac8b6b085a64bf952e138ce450f064a9661f077f5c889e9154718aa5c1ac936423e640613719a82de9772259b8bfa299bdb8bb51d973cbd6bc58838cd7d90d2cdf8bfe1651b0ab3f9b9

# Bloques

In [50]:
# block.py
import time
from copy import deepcopy

class Block():

    def __init__(self, txs, last_hash, forger, block_count):
        self.block_count = block_count
        self.txs = txs
        self.last_hash = last_hash
        self.timestamp = time.time()
        self.forger = forger
        self.signature = ""

    def to_json(self):
        data = {}
        data["blockCount"] = self.block_count
        data["lastHash"] = self.last_hash
        data["signature"] = self.signature
        data["forger"] = self.forger
        data["timestamp"] = self.timestamp
        json_txs = []
        for tx in self.txs:
            json_txs.append(tx.to_json())
        data["txs"] = json_txs
        return data

    def payload(self):
        json_representation = deepcopy(self.to_json())
        json_representation["signature"] = ""
        return json_representation

    def sign(self, signature):
        self.signature = signature

In [51]:
# Wallet.py
from Crypto.PublicKey import RSA
# from Transaction import Transaction
#from BlockchainUtils import BlockchainUtils
from Crypto.Signature import PKCS1_v1_5
#from Block import Block    

class Wallet():

    def __init__(self):
        self.key_pair = RSA.generate(2048)

    def sign(self, data):
        data_hash = BlockchainUtils.hash(data)
        signature_scheme_object = PKCS1_v1_5.new(self.key_pair)
        signature = signature_scheme_object.sign(data_hash)
        return signature.hex()

    @staticmethod
    def signature_valid(data, signature, public_key_string):
        signature = bytes.fromhex(signature)
        data_hash = BlockchainUtils.hash(data)
        public_key = RSA.importKey(public_key_string)
        signature_scheme_object = PKCS1_v1_5.new(public_key)
        signature_valid = signature_scheme_object.verify(data_hash, signature)
        return signature_valid

    def public_key_string(self):
        public_key_string = self.key_pair.publickey().exportKey('PEM').decode("utf8")
        return public_key_string

    def create_tx(self, receiver, amount, tx_type):
        transaction = Transaction(self.public_key_string(), receiver, amount, tx_type)
        signature = self.sign(transaction.payload())
        transaction.sign(signature)
        return transaction

    def create_block(self, txs, last_hash, block_count):
        block = Block(txs, last_hash, self.public_key_string(), block_count)
        signature = self.sign(block.payload())
        block.sign(signature)
        return block

In [52]:
pool

In [53]:
for pool_tx in pool.transactions:
    print(pool_tx.to_json())

{'sender_public_key': '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAruUPk0j2OniLBE0sjZ8J\n+uQytEuE/kxFO1UPgiKrQclYiR+KsCS4v5S5EPCaCSZgJDwR0LBhCX34lZC3ikje\n7V5jZ+RM+2lzFvEU2DpizorbEOjfsePObEGP7s/v4pWu48+E2mVcMuNazchLlmfS\n63aPhB/8PROvskExo6rpgVcqZuOwZs+v766blDq7zFxJXmk8S71lFzTf/cnVDpE6\ngGBaiaC00q8VjwlHrY4wdWsfymD+jeNX57I2nFr28Mv0axA4FbNTjrTUGxCrkrco\nOrRiA+K++JvqF1UcMe0Pr/Mn/7m1llSOe8zorCrLg+grcnqFy62fZ6m0VBpm353l\niwIDAQAB\n-----END PUBLIC KEY-----', 'receiver_public_key': 'maría', 'amount': 300, 'tx_type': 'TRANSFER', 'id': 'e3ae5051e88811efa554fcb3bcfce7a6', 'timestamp': 1739286010.4823148, 'signature': '8cf651191da517546af7e7432f11b3c54e7c8910664763052ac70069204bb5404adace97475db37698bd019ec40281fdb0bcb7f70fb92c9a6202f8554a2129e5bbca5229a07d828459c537ff22ad6fc2792cfc94a8d71cafa124b23c9386506389ebabb79bd66f0df1ac8b6b085a64bf952e138ce450f064a9661f077f5c889e9154718aa5c1ac936423e640613719a82de9772259b8bfa299bdb8bb51d973cbd6bc58838cd7d90d2cdf8bfe1651b0ab3f9b9

In [54]:
wallet = Wallet()
fraudulent_wallet = Wallet()

block = wallet.create_block(pool.transactions, "last_hash", 1)
signature_valid = Wallet.signature_valid(block.payload(), block.signature, fraudulent_wallet.public_key_string())

In [55]:
print(block.to_json())

{'blockCount': 1, 'lastHash': 'last_hash', 'signature': '1f10a18100df809ffd7c32a988b2685e4b9edf954ee06a687800874397dbd612d81daa6cfcf113bcfd940e187b0c5e92a3a5d1f7cc1c2e808666d96bd232870b9360e8a0f2576f0342e21b798d66a4a4dc6f800c59c146ec66788cb9eccbf629cf286b64f73311c2d353b956820d1e59589abf9282cfb7fba6ccc2b7495c8867edd72613756ec93b7ba4b1f57febcc2ede72ddbe955c8ae304fe35726cef4d8d72e5e00e90fa017be718b28f9cb0d912602d4a8f6706835cf89b3149cb682859fad03d2064abad5c1f4c273299e35c977b38696ad30c8062ad040cacba4ebfd57f015fce269ac78992d035bce4441767afc1d7f13578b5bef67deae2f4b5f576', 'forger': '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAgO716MdcwqIiyZsVXqSj\nkqyFAbopCM7+zTyXPDJnI2TuAampMPyV/TpRhoRJEoEbq3SIxhPVaHacLOcmbG/n\nK/cVG+R2LbWBC/U1Q9HjcL1RSVWK/5NvxAH9+cEMPEh/1OeRxqk/moxi5tGznnvZ\n48qJVsDL3/0iDQ6jqIcWNoI4Sr3i2DUVPL68ASX1eC1v7JuBtnVYVoBqspXz334N\nlW8DxioyeLwnPv8Pcybs1sFnmi+cl823hHh9HoV6k8jS7xqXvVaEfMBk7E+ZpAel\nESuuHVd8BuK9HXVkS19TFSZwmyDs5HUjsPqhmtGHmrVTIJGTUd/x8qfnHaw

In [56]:
signature_valid

False

In [57]:
from pprint import pprint
pprint(block.to_json())

{'blockCount': 1,
 'forger': '-----BEGIN PUBLIC KEY-----\n'
           'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAgO716MdcwqIiyZsVXqSj\n'
           'kqyFAbopCM7+zTyXPDJnI2TuAampMPyV/TpRhoRJEoEbq3SIxhPVaHacLOcmbG/n\n'
           'K/cVG+R2LbWBC/U1Q9HjcL1RSVWK/5NvxAH9+cEMPEh/1OeRxqk/moxi5tGznnvZ\n'
           '48qJVsDL3/0iDQ6jqIcWNoI4Sr3i2DUVPL68ASX1eC1v7JuBtnVYVoBqspXz334N\n'
           'lW8DxioyeLwnPv8Pcybs1sFnmi+cl823hHh9HoV6k8jS7xqXvVaEfMBk7E+ZpAel\n'
           'ESuuHVd8BuK9HXVkS19TFSZwmyDs5HUjsPqhmtGHmrVTIJGTUd/x8qfnHawXzaZA\n'
           'IwIDAQAB\n'
           '-----END PUBLIC KEY-----',
 'lastHash': 'last_hash',
 'signature': '1f10a18100df809ffd7c32a988b2685e4b9edf954ee06a687800874397dbd612d81daa6cfcf113bcfd940e187b0c5e92a3a5d1f7cc1c2e808666d96bd232870b9360e8a0f2576f0342e21b798d66a4a4dc6f800c59c146ec66788cb9eccbf629cf286b64f73311c2d353b956820d1e59589abf9282cfb7fba6ccc2b7495c8867edd72613756ec93b7ba4b1f57febcc2ede72ddbe955c8ae304fe35726cef4d8d72e5e00e90fa017be718b28f9cb0d912602d

# Cadena de bloques (Blockchain)

In [58]:
# block.py
import time
from copy import deepcopy

class Block():

    def __init__(self, txs, last_hash, forger, block_count):
        self.block_count = block_count
        self.txs = txs
        self.last_hash = last_hash
        self.timestamp = time.time()
        self.forger = forger
        self.signature = ""

    @staticmethod
    def genesis():
        genesis_block = Block([], "genesis_hash", "genesis", 0)
        genesis_block.timestamp = 0
        return genesis_block

    def to_json(self):
        data = {}
        data["blockCount"] = self.block_count
        data["lastHash"] = self.last_hash
        data["signature"] = self.signature
        data["forger"] = self.forger
        data["timestamp"] = self.timestamp
        json_txs = []
        for tx in self.txs:
            json_txs.append(tx.to_json())
        data["txs"] = json_txs
        return data

    def payload(self):
        json_representation = deepcopy(self.to_json())
        json_representation["signature"] = ""
        return json_representation

    def sign(self, signature):
        self.signature = signature
        

In [59]:
# Blockchain.py
# from BlockchainUtils import BlockchainUtils
# from Block import Block

class Blockchain():

    def __init__(self):
        self.blocks = [Block.genesis()]

    def add_block(self, block):
        self.blocks.append(block)

    def to_json(self):
        data = {}
        json_blocks = []
        for block in self.blocks:
            json_blocks.append(block.to_json())
        data["blocks"] = json_blocks
        return data

    def block_count_valid(self, block):
        if self.blocks[-1].block_count == block.block_count - 1:
            return True
        else:
            return False

    def last_block_hash_valid(self, block):
        latest_blockchain_block_hash = BlockchainUtils.hash(self.blocks[-1].payload()).hexdigest()
        if latest_blockchain_block_hash == block.last_hash:
            return True
        else:
            return False

In [60]:
# main.py

#from pprint import pprint
#from Blockchain import Blockchain

blockchain = Blockchain()

last_hash = BlockchainUtils.hash(blockchain.blocks[-1].payload()).hexdigest()
block_count = blockchain.blocks[-1].block_count + 1

block = wallet.create_block(pool.transactions, last_hash, block_count)

if not blockchain.last_block_hash_valid(block):
    print("last block hash not valid")
if not blockchain.block_count_valid(block):
    print("block count is not valid")

if blockchain.last_block_hash_valid(block) and blockchain.block_count_valid(block):
    blockchain.add_block(block)

pprint(blockchain.to_json())

{'blocks': [{'blockCount': 0,
             'forger': 'genesis',
             'lastHash': 'genesis_hash',
             'signature': '',
             'timestamp': 0,
             'txs': []},
            {'blockCount': 1,
             'forger': '-----BEGIN PUBLIC KEY-----\n'
                       'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAgO716MdcwqIiyZsVXqSj\n'
                       'kqyFAbopCM7+zTyXPDJnI2TuAampMPyV/TpRhoRJEoEbq3SIxhPVaHacLOcmbG/n\n'
                       'K/cVG+R2LbWBC/U1Q9HjcL1RSVWK/5NvxAH9+cEMPEh/1OeRxqk/moxi5tGznnvZ\n'
                       '48qJVsDL3/0iDQ6jqIcWNoI4Sr3i2DUVPL68ASX1eC1v7JuBtnVYVoBqspXz334N\n'
                       'lW8DxioyeLwnPv8Pcybs1sFnmi+cl823hHh9HoV6k8jS7xqXvVaEfMBk7E+ZpAel\n'
                       'ESuuHVd8BuK9HXVkS19TFSZwmyDs5HUjsPqhmtGHmrVTIJGTUd/x8qfnHawXzaZA\n'
                       'IwIDAQAB\n'
                       '-----END PUBLIC KEY-----',
             'lastHash': '698015b9c44cdd0f8801feeb72f159d39e48710e18012c433acfac83127357

In [61]:
last_hash = BlockchainUtils.hash(blockchain.blocks[-1].payload()).hexdigest()
block_count = blockchain.blocks[-1].block_count + 1

block = wallet.create_block(pool.transactions, last_hash, block_count)

if not blockchain.last_block_hash_valid(block):
    print("last block hash not valid")
if not blockchain.block_count_valid(block):
    print("block count is not valid")

if blockchain.last_block_hash_valid(block) and blockchain.block_count_valid(block):
    blockchain.add_block(block)

pprint(blockchain.to_json())

{'blocks': [{'blockCount': 0,
             'forger': 'genesis',
             'lastHash': 'genesis_hash',
             'signature': '',
             'timestamp': 0,
             'txs': []},
            {'blockCount': 1,
             'forger': '-----BEGIN PUBLIC KEY-----\n'
                       'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAgO716MdcwqIiyZsVXqSj\n'
                       'kqyFAbopCM7+zTyXPDJnI2TuAampMPyV/TpRhoRJEoEbq3SIxhPVaHacLOcmbG/n\n'
                       'K/cVG+R2LbWBC/U1Q9HjcL1RSVWK/5NvxAH9+cEMPEh/1OeRxqk/moxi5tGznnvZ\n'
                       '48qJVsDL3/0iDQ6jqIcWNoI4Sr3i2DUVPL68ASX1eC1v7JuBtnVYVoBqspXz334N\n'
                       'lW8DxioyeLwnPv8Pcybs1sFnmi+cl823hHh9HoV6k8jS7xqXvVaEfMBk7E+ZpAel\n'
                       'ESuuHVd8BuK9HXVkS19TFSZwmyDs5HUjsPqhmtGHmrVTIJGTUd/x8qfnHawXzaZA\n'
                       'IwIDAQAB\n'
                       '-----END PUBLIC KEY-----',
             'lastHash': '698015b9c44cdd0f8801feeb72f159d39e48710e18012c433acfac83127357

In [62]:
last_hash = BlockchainUtils.hash(blockchain.blocks[-1].payload()).hexdigest()
block_count = blockchain.blocks[-1].block_count + 1

block = wallet.create_block(pool.transactions, last_hash, block_count)

if not blockchain.last_block_hash_valid(block):
    print("last block hash not valid")
if not blockchain.block_count_valid(block):
    print("block count is not valid")

if blockchain.last_block_hash_valid(block) and blockchain.block_count_valid(block):
    blockchain.add_block(block)

pprint(blockchain.to_json())

{'blocks': [{'blockCount': 0,
             'forger': 'genesis',
             'lastHash': 'genesis_hash',
             'signature': '',
             'timestamp': 0,
             'txs': []},
            {'blockCount': 1,
             'forger': '-----BEGIN PUBLIC KEY-----\n'
                       'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAgO716MdcwqIiyZsVXqSj\n'
                       'kqyFAbopCM7+zTyXPDJnI2TuAampMPyV/TpRhoRJEoEbq3SIxhPVaHacLOcmbG/n\n'
                       'K/cVG+R2LbWBC/U1Q9HjcL1RSVWK/5NvxAH9+cEMPEh/1OeRxqk/moxi5tGznnvZ\n'
                       '48qJVsDL3/0iDQ6jqIcWNoI4Sr3i2DUVPL68ASX1eC1v7JuBtnVYVoBqspXz334N\n'
                       'lW8DxioyeLwnPv8Pcybs1sFnmi+cl823hHh9HoV6k8jS7xqXvVaEfMBk7E+ZpAel\n'
                       'ESuuHVd8BuK9HXVkS19TFSZwmyDs5HUjsPqhmtGHmrVTIJGTUd/x8qfnHawXzaZA\n'
                       'IwIDAQAB\n'
                       '-----END PUBLIC KEY-----',
             'lastHash': '698015b9c44cdd0f8801feeb72f159d39e48710e18012c433acfac83127357

In [63]:
last_hash = BlockchainUtils.hash(blockchain.blocks[-1].payload()).hexdigest()
block_count = blockchain.blocks[-1].block_count + 1

block = wallet.create_block(pool.transactions, last_hash, block_count)

if not blockchain.last_block_hash_valid(block):
    print("last block hash not valid")
if not blockchain.block_count_valid(block):
    print("block count is not valid")

if blockchain.last_block_hash_valid(block) and blockchain.block_count_valid(block):
    blockchain.add_block(block)

pprint(blockchain.to_json())

{'blocks': [{'blockCount': 0,
             'forger': 'genesis',
             'lastHash': 'genesis_hash',
             'signature': '',
             'timestamp': 0,
             'txs': []},
            {'blockCount': 1,
             'forger': '-----BEGIN PUBLIC KEY-----\n'
                       'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAgO716MdcwqIiyZsVXqSj\n'
                       'kqyFAbopCM7+zTyXPDJnI2TuAampMPyV/TpRhoRJEoEbq3SIxhPVaHacLOcmbG/n\n'
                       'K/cVG+R2LbWBC/U1Q9HjcL1RSVWK/5NvxAH9+cEMPEh/1OeRxqk/moxi5tGznnvZ\n'
                       '48qJVsDL3/0iDQ6jqIcWNoI4Sr3i2DUVPL68ASX1eC1v7JuBtnVYVoBqspXz334N\n'
                       'lW8DxioyeLwnPv8Pcybs1sFnmi+cl823hHh9HoV6k8jS7xqXvVaEfMBk7E+ZpAel\n'
                       'ESuuHVd8BuK9HXVkS19TFSZwmyDs5HUjsPqhmtGHmrVTIJGTUd/x8qfnHawXzaZA\n'
                       'IwIDAQAB\n'
                       '-----END PUBLIC KEY-----',
             'lastHash': '698015b9c44cdd0f8801feeb72f159d39e48710e18012c433acfac83127357

In [64]:
last_hash = BlockchainUtils.hash(blockchain.blocks[-1].payload()).hexdigest()
block_count = blockchain.blocks[-1].block_count + 1

block = wallet.create_block(pool.transactions, last_hash, block_count)

if not blockchain.last_block_hash_valid(block):
    print("last block hash not valid")
if not blockchain.block_count_valid(block):
    print("block count is not valid")

if blockchain.last_block_hash_valid(block) and blockchain.block_count_valid(block):
    blockchain.add_block(block)

pprint(blockchain.to_json())

{'blocks': [{'blockCount': 0,
             'forger': 'genesis',
             'lastHash': 'genesis_hash',
             'signature': '',
             'timestamp': 0,
             'txs': []},
            {'blockCount': 1,
             'forger': '-----BEGIN PUBLIC KEY-----\n'
                       'MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAgO716MdcwqIiyZsVXqSj\n'
                       'kqyFAbopCM7+zTyXPDJnI2TuAampMPyV/TpRhoRJEoEbq3SIxhPVaHacLOcmbG/n\n'
                       'K/cVG+R2LbWBC/U1Q9HjcL1RSVWK/5NvxAH9+cEMPEh/1OeRxqk/moxi5tGznnvZ\n'
                       '48qJVsDL3/0iDQ6jqIcWNoI4Sr3i2DUVPL68ASX1eC1v7JuBtnVYVoBqspXz334N\n'
                       'lW8DxioyeLwnPv8Pcybs1sFnmi+cl823hHh9HoV6k8jS7xqXvVaEfMBk7E+ZpAel\n'
                       'ESuuHVd8BuK9HXVkS19TFSZwmyDs5HUjsPqhmtGHmrVTIJGTUd/x8qfnHawXzaZA\n'
                       'IwIDAQAB\n'
                       '-----END PUBLIC KEY-----',
             'lastHash': '698015b9c44cdd0f8801feeb72f159d39e48710e18012c433acfac83127357